In [ ]:
import os

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import bilby
import corner
import glob
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import numpy.lib.recfunctions as rfn
import tqdm
import seaborn as sns
from scipy import stats

from importlib import reload

import thesis_utils
from thesis_utils.gw import get_cbc_parameter_labels

reload(thesis_utils.gw.utils)
from thesis_utils.plotting import (
    set_plotting,
    save_figure,
    get_default_corner_kwargs,
    get_default_figsize,
)

set_plotting()

In [ ]:
posterior_path = "/home/michael.williams/public_html/nessai-gwtc/O3/GW190412/samples/posterior_samples.h5"
samples = {}
with h5py.File(posterior_path, "r") as f:
    samples["gwtc2p1"] = f["GWTC-2_1/posterior_samples"][()]
    samples["nessai"] = f["nessai/posterior_samples"][()]

In [ ]:
parameters = ["chirp_mass", "mass_ratio"]
colours = {
    "gwtc2p1": "k",
    "nessai": "C0",
}
labels = {
    "gwtc2p1": "GWTC-2.1",
    "nessai": r"\texttt{nessai}",
}

corner_kwargs = get_default_corner_kwargs()
corner_kwargs["plot_density"] = True
corner_kwargs["plot_datapoints"] = False
corner_kwargs["no_fill_contours"] = True
corner_kwargs["show_titles"] = False
corner_kwargs.pop("fill_contours")
corner_kwargs["levels"] = (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.0))
corner_kwargs["quantiles"] = [0.16, 0.84]
corner_kwargs["smooth"] = 0.9
corner_kwargs["bins"] = 32

figsize = get_default_figsize()
figsize[0] = figsize[1]
fig = plt.figure(figsize=figsize)

legend_elements = []

corner_kwargs["hist_kwargs"]["ls"] = "-"
for i, (key, data) in enumerate(samples.items()):
    corner_kwargs["color"] = colours.get(key)
    corner_kwargs["hist_kwargs"]["color"] = colours.get(key)
    fig = corner.corner(
        rfn.structured_to_unstructured(data[parameters]),
        fig=fig,
        labels=get_cbc_parameter_labels(parameters, units=True),
        labelpad=-0.1,
        **corner_kwargs,
    )
    legend_elements.append(
        Line2D([0], [0], color=colours.get(key), ls="-", label=labels.get(key))
    )


fig.legend(
    handles=legend_elements,
    bbox_to_anchor=(0.75, 0.8),
    loc="center",
    fontsize=12,
)
plt.show()
save_figure(fig, "GW190412_posterior", "figures/GW190412/")

In [ ]:
nessai_result_path = "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/O3/outdir_GW190412_IMRPhenomXPHM/result/GW190412_IMRPhenomXPHM_data0_1239082262-17041_analysis_H1L1V1_par*_nessai/result.hdf5"
indices = []
nessai_result_files = glob.glob(nessai_result_path)
for rf in nessai_result_files:
    with h5py.File(rf, "r") as f:
        indices.append(f["insertion_indices"][()])

In [ ]:
figsize = 0.6 * get_default_figsize()
fig = plt.figure(figsize=figsize)
# plt.axhline(0, color="grey")
x = np.arange(0, 1000, 1)
analytic = x / x[-1]
n = np.mean([len(idx) for idx in indices])
c = sns.color_palette("Blues", n_colors=3)[1:]
for i, idx in enumerate(indices):
    _, counts = np.unique(idx, return_counts=True)
    estimated = np.cumsum(counts) / len(idx)
    plt.plot(analytic - estimated, ls="-", c=c[i])

for ci in [0.5, 0.95, 0.997]:
    bound = (1 - ci) / 2
    bound_values = stats.binom.ppf(1 - bound, n, analytic) / n
    lower = bound_values - analytic
    upper = analytic - bound_values
    upper[0] = 0
    upper[-1] = 0
    lower[0] = 0
    lower[-1] = 0

    plt.fill_between(x, lower, upper, color="grey", alpha=0.2)

plt.xlim(0, 1000)
plt.xlabel("Insertion index")
plt.ylabel(r"Analytic CMF - Empirical CMF")
plt.show()
save_figure(fig, "GW190412_insertion_indices", "figures/GW190412/")
fig.savefig("figures/GW190412/insertion_indices.png", dpi=200, bbox_inches="tight")